In [14]:
!pwd

/data0/jiahui/github/PepBenchmark/examples/property_pred/lm-based


In [ ]:
import os
import json
import pandas as pd
from tqdm import tqdm

# 参数组合
split_indices = ["random1", "random2", "random3", "random4", "random5"]
split_types = ["Random_split", "Homology_based_split"]
model_map = {
    "prot_bert_bfd": "Rostlab/prot_bert_bfd",
    "esm2_150M": "facebook/esm2_t30_150M_UR50D",
    "esm2_650M": "facebook/esm2_t33_650M_UR50D",
    "dplm_150m":"airkingbd/dplm_150m",
    "dplm_650m":"airkingbd/dplm_650m"
}
model_names = model_map.values()
# 汇总结果
results = []
task="AF_APML"
# task = "Nonfouling"
# 遍历所有组合
for split_type in split_types:
    for split_index in split_indices:
        for model_name in model_names:
     
            result_dir = os.path.join("checkpoints",task, split_type, model_name,split_index)

            if not os.path.exists(result_dir):
                print(f"Directory {result_dir} does not exist, skipping...")
                continue
            
            row = {
                "split_type": split_type,
                "split_index": split_index,
                "model_name": model_name,
            }
            mode = "test"
            metric_file = os.path.join(result_dir, f"{mode}_metrics.csv")
            res = pd.read_csv(metric_file)
            # 将row与res的第一行拼接
            row.update(res.iloc[0].to_dict())
    
                
            results.append(row)

# 创建DataFrame
df = pd.DataFrame(results)

# 保存为CSV
df.to_csv("./results/all_results_summary.csv", index=False)

# 显示部分结果
print(df.head())


     split_type split_index                    model_name  test_test_loss  \
0  Random_split     random1         Rostlab/prot_bert_bfd        0.405906   
1  Random_split     random1  facebook/esm2_t30_150M_UR50D        0.449092   
2  Random_split     random1  facebook/esm2_t33_650M_UR50D        0.425968   
3  Random_split     random1           airkingbd/dplm_150m        0.457380   
4  Random_split     random1           airkingbd/dplm_650m        0.309182   

   test_test_accuracy  test_test_f1  test_test_precision  test_test_recall  \
0            0.845679      0.889868             0.821138          0.971154   
1            0.888889      0.914286             0.905660          0.923077   
2            0.901235      0.925234             0.900000          0.951923   
3            0.913580      0.933962             0.916667          0.951923   
4            0.932099      0.946860             0.951456          0.942308   

   test_test_micro_f1  test_test_macro_f1  test_test_roc_auc  \
0   

In [44]:
df

,split_type,split_index,model_name,test_test_loss,test_test_accuracy,test_test_f1,test_test_precision,test_test_recall,test_test_micro_f1,test_test_macro_f1,test_test_roc_auc,test_test_runtime,test_test_samples_per_second,test_test_steps_per_second
0,Random_split,random1,Rostlab/prot_bert_bfd,0.405906,0.845679,0.889868,0.821138,0.971154,0.845679,0.816068,0.795922,0.6634,244.212,4.522
1,Random_split,random1,facebook/esm2_t30_150M_UR50D,0.449092,0.888889,0.914286,0.905660,0.923077,0.888889,0.878195,0.875332,0.7228,224.142,4.151
2,Random_split,random1,facebook/esm2_t33_650M_UR50D,0.425968,0.901235,0.925234,0.900000,0.951923,0.901235,0.889890,0.881134,0.8097,200.068,3.705
3,Random_split,random1,airkingbd/dplm_150m,0.457380,0.913580,0.933962,0.916667,0.951923,0.913580,0.904481,0.898375,0.7146,226.704,4.198
4,Random_split,random1,airkingbd/dplm_650m,0.309182,0.932099,0.946860,0.951456,0.942308,0.932099,0.926421,0.928050,0.8149,198.793,3.681
5,Random_split,random2,Rostlab/prot_bert_bfd,0.610119,0.790123,0.831683,0.777778,0.893617,0.790123,0.776497,0.770338,0.5054,320.544,5.936
6,Random_split,random2,facebook/esm2_t30_150M_UR50D,0.334801,0.913580,0.925532,0.925532,0.925532,0.913580,0.911295,0.911295,0.7140,226.890,4.202
7,Random_split,random2,facebook/esm2_t33_650M_UR50D,0.361244,0.895062,0.907104,0.932584,0.882979,0.895062,0.893268,0.897372,0.8064,200.886,3.720
8,Random_split,random2,airkingbd/dplm_150m,0.339165,0.913580,0.924731,0.934783,0.914894,0.913580,0.911641,0.913329,0.7118,227.591,4.215
9,Random_split,random2,airkingbd/dplm_650m,0.392319,0.882716,0.898396,0.903226,0.893617,0.882716,0.879855,0.880632,0.9977,162.381,3.007


In [45]:
# 按照split_type和split_index进行分组，并只对数值型列计算平均值和方差
numeric_cols = df.select_dtypes(include='number').columns
grouped_df = df.groupby(["split_type", "model_name"])[numeric_cols].agg(['mean', 'std']).reset_index()
grouped_df[["model_name", "split_type","test_test_roc_auc"]]

model_name            split_type test_test_roc_auc  \
                                                                   mean   
0         Rostlab/prot_bert_bfd  Homology_based_split          0.665371   
1           airkingbd/dplm_150m  Homology_based_split          0.759730   
2           airkingbd/dplm_650m  Homology_based_split          0.720680   
3  facebook/esm2_t30_150M_UR50D  Homology_based_split          0.749560   
4  facebook/esm2_t33_650M_UR50D  Homology_based_split          0.743329   
5         Rostlab/prot_bert_bfd          Random_split          0.789410   
6           airkingbd/dplm_150m          Random_split          0.878004   
7           airkingbd/dplm_650m          Random_split          0.890968   
8  facebook/esm2_t30_150M_UR50D          Random_split          0.866896   
9  facebook/esm2_t33_650M_UR50D          Random_split          0.883308   

             
        std  
0  0.028420  
1  0.041253  
2  0.057687  
3  0.035300  
4  0.025292  
5  0.014827  
6  0.029685  
7  0.032135  
8  0.032909  
9  0.019146

In [55]:

df_filtered = df[(df['split_type'] == 'Random_split') & (df['split_index'] == 'random1')]




df_filtered[["model_name", "test_test_roc_auc","test_test_accuracy","test_test_precision","test_test_recall","test_test_f1"]].sort_values(by="test_test_roc_auc", ascending=False)

,model_name,test_test_roc_auc,test_test_accuracy,test_test_precision,test_test_recall,test_test_f1
4,airkingbd/dplm_650m,0.928050,0.932099,0.951456,0.942308,0.946860
3,airkingbd/dplm_150m,0.898375,0.913580,0.916667,0.951923,0.933962
2,facebook/esm2_t33_650M_UR50D,0.881134,0.901235,0.900000,0.951923,0.925234
1,facebook/esm2_t30_150M_UR50D,0.875332,0.888889,0.905660,0.923077,0.914286
0,Rostlab/prot_bert_bfd,0.795922,0.845679,0.821138,0.971154,0.889868


In [56]:

df_filtered = df[(df['split_type'] == 'Homology_based_split') & (df['split_index'] == 'random1')]

df_filtered[["model_name", "test_test_roc_auc","test_test_accuracy","test_test_precision","test_test_recall","test_test_f1"]].sort_values(by="test_test_roc_auc", ascending=False)



,model_name,test_test_roc_auc,test_test_accuracy,test_test_precision,test_test_recall,test_test_f1
28,airkingbd/dplm_150m,0.780952,0.801242,0.714286,0.714286,0.714286
29,airkingbd/dplm_650m,0.746429,0.788820,0.739130,0.607143,0.666667
27,facebook/esm2_t33_650M_UR50D,0.722619,0.714286,0.567568,0.750000,0.646154
26,facebook/esm2_t30_150M_UR50D,0.689286,0.670807,0.518519,0.750000,0.613139
25,Rostlab/prot_bert_bfd,0.639286,0.627329,0.475000,0.678571,0.558824
